# N-gram LM
- ref: chapter3 of SLP, lecture 1 of cmu 11-711 spring 2024

## I. 语言模型
1. 定义形式：$P(X)= P(x_1, x_2, ···,x_{I-1},x_I) $ \
**Chain rule：**$P(X)= \prod_{i=1}^{I}P(x_i|x_1, x_2, ···,x_{i-1}) $  ，I是sentence长度
2. <font color=red>求解语言模型的关键是估计$P(x_i|x_1, x_2, ···,x_{i-1})$</font>
3. <font color=green>**最简单的估计方法是n-gram models：**</font>
$P(x_i|x_1, ···, x_{i-1}) \approx  P(x_i|x_{i-n+1},···, x_{i-1})$

- **<font color=norange>注：chain rule在数学上成立，但用chain rule计算句子发生的概率却不符合现实。</font>**
  - 理解：
    1. 从统计的角度，对sentence x发生的概率进行分解，那么$P(X)= \prod_{i=1}^{I}P(x_i|x_1, x_2, ···,x_{i-1})$一定成立。
    2. 从语言实践角度，人们造句并不按概率，而是根据表述的目的来选择用语。也就是说，chain rule虽然数值上正确，但是它并不是刻画语言生成过程的合理方式。这种不匹配性也体现在无法正确计算$P(x_i|x_1, x_2, ···,x_{i-1})$。直接用统计的方式count历史上语料中的词频是不合理的，因为人们每天都在按照自己的需要讲话，很多表达在历史上都是zero count。
  - <font color=green>n-gram的出发点是，反正都无法计算真正的$P(x_i|x_1, x_2, ···,x_{i-1})$，那就用一种粗略的方式来approximate。它引入了很强的"假设"，$x_i$只受前面n-1个词影响。虽然这个假设明显不正确，但在特定语境下该模型有使用价值。比如：垃圾邮件检测、情绪分类。</font>
    - 因为假设前提与语言分布的实际情况相差大，用n-gram解决不了语言生成的问题
    - 但在文本分类问题中，不需要正确地生成句子，只要抓住目标类型文本中明显的feature就可以做分类。
      - 以垃圾邮件为例，通常邮件中会出现推销、奖励相关的特定词段，tri-gram或者dual-gram已经可以抓这些词段，从而在计算sentence Probability的时候体现出来。

## II. N-gram models

### II.1 常见的N-gram models

#### II.1.1 最简单的语言模型：count-based unigram model
**特点：**
1. naive bayesian假设(条件独立)：$P(x_i|x_1, x_2, ···, x_{i-1})=P(x_i)$ \
此时，得到的语言模型是unigram model：$P(X)= \prod_{i=1}^{I}P(x_i) $
2. 模型训练过程中，使用的求解$P(x_i)$的方法：MLE
$$\begin{align}
L(θ) & = \prod_{i} P(x)^{C(x_i)}\\
等价于： & log L(θ) = \sum_{i} C(x_i) * log P(x_i)\\
约束：& \sum_{i}P(x_i) = 1
\end{align}$$
3. 优化结果是count-based estimate：
$$P_{MLE}(x_i)=\frac{C_{train}(x_i)}{Σ_jC_{train}(x_j)}$$

#### II.1.2 n-gram model
- **特点：**
  1. 假设：每个词出现的概率与前n-1个位置的词语有关。
$$\begin{align}
P(x_i|x_1, x_2, ···, x_{i-1}) & = P(x_i|x_{i-n+1}, x_{i-n+2}, ···, x_{i-1}) \\
& = P(x_i|x_{i-n+1}:x_{i-1})
\end{align}$$ 
  2. 目标函数：MLE
  3. 优化结果是count-based estimate： 
$$\begin{align}
P(x_i|x_{i-n+1}:x_{i-1}) & = \frac{C_{train}(x_{i-n+1}:x_i)}{Σ_{j}C_{train}(x_{i-n+1}:x_{i-1},x_j)}\\
& = \frac{C_{train}(x_{i-n+1}:x_i)}{C_{train}(x_{i-n+1}:x_{i-1})}
\end{align}$$

- **count table：** 
  - 例：取n=2时，用Berkeley Restaurant Project的数据(|V|=1446)给8个words统计count table。下图是一部分，行列都不完整
<img src="../pics/BerkeleyRestaurantCountTable.png" style="width: 400px">
  - 将count table中的count转化为probability：每行加总为1
<img src="../pics/BerkeleyRestaurantProbTable.png" style="width: 400px">
  - count table的典型特征：<font color=blue>table中有很多0，非常sparse</font>
  - count table normalize之后拿到的probability可以反映语料的信息：
    - 语言本身的语义信息，比如：动词后面通常是名词或形容词
    - 语言内容中体现的文化信息，比如：上表中，大比例的食物是中餐而不是西餐，说明语料涉及的人们更喜欢中餐

### II.2 N-gram model中的问题
#### II.2.1 N-gram model训练结果中的两个典型特征 
1. <font color=orange>model的N越大，训练好的模型用sample method得到的句子的连贯性看起来会越好。</font>但这并不是模型生成语句的能力变强，而是model记忆了trainning set的词组，这种记忆没有泛化能力。
     - 在莎士比亚文本语料（88万个word）基础上训练unigram到four-gram model，然后在训练好的模型上用sample methods得到下图结果。
     <img src="../pics/N-gramOnShakespear.png" style="width: 500px"> 
2. <font color=orange>训练得到的probability通常包含了训练语料中的具体信息。</font>这同样只是model记忆了trainning set的体现。一旦使用场景的语言特征和训练语料的差异较大，那么模型效果就会很差。
     - <font color=green>比如以four-gram为例，很多四个词的词组在语料库中重复出现的概率不高，当词组仅出现一次的时候，模型learn到的该组合的$P(x_i|context_i)=1$，生成内容时，这些词组就是被记忆的内容。</font>
     - 比较下图中用WSJ语料（4000万个word）上训练的uni-gram到tri-gram model做sample的结果与莎士比亚sample结果可以看到两者的语言风格截然不同。
     <img src="../pics/N-gramWSJ.png" style="filter: grayscale(100%); width: 500px"> 

#### II.2.2 上述特征背后的sparsity问题
1. **造成上述现象的原因是N-gram model有Sparsity的问题**
   - 随着N和|V|的增加，训练结果中$P(x_i|x_{i-n+1}:x_{i-1})=0$的占比越来越高，n-gram probability matrix越来越sparse。</font> 
   - <font color=blue>**N,|V|与probability matrice大小的关系：** $matrice中的elements数量=|V|^{N}$ </font> 
N=2，|V|=29k时，$|V|^2\approx 8e8$
N=3，|V|=29k时，$|V|^3\approx 2e13$
N=4，|V|=29k时，$|V|^4\approx 7e17$ 
   - <font color=orange>一般的语料库很难达到让probability matrx变得dense所需的大小。</font>莎士比亚语料的词典长度|V|是29k，corpus长度大约9e5；WSJ语料|V|是20k，corpus长度大约4e7。而这些corpus去掉重复的N-gram后，覆盖matrix的比例更小。
   - 在句子生成场景，这种sparsity意味着，如果用sample method来generate sentence，那么很多时候会抽到corpus中的原句。因为probability matrix中很多row里面只有1个element的概率非0,且为1。这就解释了模型能记忆原文的原因。

2. <font color=blue>**sparsity引发的模型应用问题和解决方向** </font> 
   - (1)很多实际概率不为0的词组的Probability被估计为0，这会降低模型在应用中的效果。
      - 比如：training set中只有denied the rumours/report，test set中如果出现denied the offer，概率就是0。这非常不合理。
   - (2)由于perplexity的公式是$P(X)^{\frac{1}{N}}$，一旦$P(X)=0$，Perplexity的值无法计算.
   - 解决思路：
     - 一是想办法把zero probabilities调成非零。<font color=red>但这种方法并不会真正提高模型的泛化能力</font>
     - 二是根据应用场景来选择与之match的语料库。当然，这也不会提升模型本身的能力。但是是工程上好用的方式。这时model即使只是记忆了trainning set，它记忆的知识可能也能满足应用场景的需求。

### II.3 处理zero-probability
#### II.3.1 两类zero-probabilty
- zero-probabilty有两类，一类是unknown words，一类是出现在novel test set context的n-gram词组。各有不同的处理方式。
  - <font color=blue>**unknown words**</font>：n-gram中出现了词典中没有的词。
  - <font color=blue>**novel n-gram词组**</font>：词典中有n-gram中出现的单个word，但是他们的组合方式在trainning set中没有出现过。这种情况一般用smoothing，也称为discounting。思路是划分一部分概率给novel n-gram，使其最低值>0. 
- 还有一种实际更常用的方式是，不用words作为基本单位来训练模型，而是用characters或者sub-words。

#### II.3.2 处理unknown words
1. **方法1：在词典中加上\<UNK>，所有unknown words都当做\<UNK>处理。**
   - <font color=orange>情况1： 如果原本有一个vocabulary：</font>
一种最简单但不是非常合理的处理方式是，在模型训练时，trainning set中出现，但没有出现在vocabulary中的词都转化为\<UNK>，并将\<UNK>当做常规词一样处理。test的时候，也将所有不在vocabulary中的词转化为\<UNK>再将\<UNK>当做一个常规词处理。
   - <font color=orange>情况2： 如果原本没有一个vocabulary：</font>
用trainning set建一个implicit vocabulary。具体方法是：将trainning set中出现频率低于一定数量的词语都转化成\<UNK>，其他words则每个都作为一个单独的word放入vocabulary。训练和测试则和方法1中相同。 
2. <font color=blue>要注意的是，处理\<UNK>的具体方式会影响perplexity的结果。因此，要比较两个model之间的perplexity的时候，一定要确保他们用的vocabulary是一样的。</font> 
   - eg: 将大量word都处理成\<UNK>可以降低vocabulary的词汇量。模型可以通过选择一个很小的vocabulary来降低perplexity。这时候\<UNK>被赋予的probability会很高。
3. **方法2：用sub-words/characters，确保所有token都在vocabulary中。**

#### II.3.3 处理novel n-gram
- **问题：** test时有的n-gram组合在trainning set中没有出现过，但是词典中有n-gram中所包含的每一个word。由于在trainning data中没有，直接用count-based MLE会得到count=0，$P(x_{unk})=0$，导致test时出现：L(θ)=0
- **处理思路：** 需要一种分布，对所有可能的words所赋予的概率都大于0.
- **方法**：
  - 最常用的处理方法：character/subword-based model
  - smoothing
  - uniform distribution：基本不用
     - 思路：在训练模型时，假设|V|固定，并让所有的unknown words的概率为$P_{unk}=\frac{1}{N_{|V|}}$

### II.4 几种smoothing方法

#### II.4.1 第一种：Laplace smoothing(add-one smoothing)
- <font color=blue>思想：在训练模型时，给所有n-gram组合的count都加1，使得unknown n-grams的count=1。</font>
$$adjusted\ P(x_i)=\frac{C(x_i)+1}{Σ_{\tilde{x}}(C(\tilde{x})+1)} = \frac{C(x_i)+1}{Σ_{\tilde{x}}C(\tilde{x})+|V|}$$
- 问题：
  - i. unknown n-grams的数量会很大，会导致$|V| \to \infty$,使$adjusted\ P(x_i)\to 0$ 
  - ii. 如果unknown n-grams的数量大，也会过度扭曲原分布。
    - <font color=orange>扭曲程度分析思路：以bi-gram为例</font> 
      - 找一组新的n-gram counts$\ C(x_{i-1}, x_i)^{'}$，在不改变$C(x_{i-1})$的条件下，满足$P_{MLE}^{'}(x_i|x_{i-1})=P_{add-1}(x_i|x_{i-1})$。此时，比较$C(x_{i-1}, x_i)^{'}$和$C(x_{i-1}, x_i)$的差异。
$${\begin{align} 
P_{MLE}^{'}(x_i|x_{i-1}) & =\frac{C(x_{i-1}, x_i)^{'}}{C(x_{i-1})}=P_{add-1}(x_i|x_{i-1}) \\
C(x_{i-1}, x_i)^{'}& = P_{add-1}(x_i|x_{i-1}) * C(x_{i-1}) \\
& = \frac{C(x_{i-1}, x_i)+1}{C(x_{i-1})+|V|} * C(x_{i-1}) \\
& = \frac{1}{1+\frac{|V|}{C(x_{i-1})}} * [C(x_{i-1}, x_i) +1 ] \\
& \approx \frac{1}{1+\frac{|V|}{C(x_{i-1})}} * C(x_{i-1}, x_i)
\end{align}}$$
      - <font color=red>此时，$C(x_{i-1}, x_i)^{'}$和$C(x_{i-1}, x_i)$的差异就体现了smoothing对语料的扭曲效果。从公式可见，差异由|V|决定。|V|越大，probability matrix被扭曲的程度越大。</font> 
- <font color=green>**所以laplace smoothing一般只用在有零概率问题，但是不严重(|V|较小)的场景，比如unigram中，n-gram不用。**</font>

#### II.4.2 第二种：add-k smoothing 
- 思路：和Laplace基本一样是转移一部分probability mass给0概率事件，只是转移给0概率事件的probability mass比laplace中要少。这样可以减轻Laplace中的扭曲问题。
- 实现方式：Laplace smoothing在训练模型时，给所有n-gram组合的count都加1。而这里只加k，且k<1。在应用中k是超参数。
  $$adjusted\ P(x_i)=\frac{C(x_i)+k}{Σ_{\tilde{x}}C(\tilde{x})+k|V|}$$
- 实践情况：对LM而言的效果并没有比Laplace好多少，所以实际也用得少

#### II.4.3 第三种：backoff and interpolation
- 思路：当$P(x_i|x_{i-n+1}:x_{i-1})=0$时，用$P(x_i|x_{i-n+2}:x_{i-1})$来代替，也就是backoff to short sequence。
- 扩展到interpolation的思路：同时用unigram到n-gram的加权概率来做n-gram probability。
  - eg：$P^{'}(x_i|x_{i-2},x_{i-1})=λ_1*P(x_i)+λ_2*P(x_i|x_{i-1})+(1-λ_1-λ_2)*P(x_i|x_{i-2},x_{i-1})$

#### II.4.4 第四种：Kneser-Ney smoothing
- 这是2013年前在实际应用和研究领域被广泛使用的模型，NN+word2vec出现后也没有马上改变。主要原因是耗用的计算量比NN少，因此在应用中能够处理比NN更大的训练样本集。
- 思路：
- 模型：(略)